In [1]:
from gensim.models import KeyedVectors
import pickle

D:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
D:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
path = r"D:\UofT Courses\Computational Models of Semantic Change\Assignment 1\\"
path_analogy = path+"word-test.v1.txt"
path_pair = path+"word_rg64_pairs.txt"

In [3]:
print("Started Loading")
path_google_vec = "GoogleNews-vectors-negative300.bin"
google_model = KeyedVectors.load_word2vec_format(path+path_google_vec, binary=True)
print("Done")

Started Loading
Done


In [4]:
with open(
        path+"lsa_word_vec_with_word_index300_final1.pkl",
        'rb'
) as f:
    lsa_mat_300, word_to_index = pickle.load(f)
    
lsa_model = KeyedVectors(300)
word_list = [item[0].upper() for item in sorted(word_to_index.items(), key=lambda item: item[1])]
lsa_model.add_vectors(word_list, lsa_mat_300)

#### Step 3 Questions
Evaluate the pair-wise similarity score

In [5]:
path_pairs = "word_rg64_pairs.txt"

pair_score_dict = {}
with open(path+path_pairs) as f:
    for line in f.readlines():
        w1, w2, score = line.split()
        pair_score_dict[(w1.lower(),w2.lower())] = float(score)
        
        # Sanity Check
        if w1.lower() not in word_to_index:
            print(1,w1,w2)
        if w2.lower() not in word_to_index:
            print(2,w1,w2)

In [6]:
p_corr, _, ratio = lsa_model.evaluate_word_pairs(path+path_pairs, delimiter=' ')

In [7]:
p_corr

(0.3117160089716393, 0.011481445001381076)

In [8]:
ratio

0.0

In [9]:
p_corr, _, _ = google_model.evaluate_word_pairs(path+path_pairs, delimiter=' ')

In [10]:
p_corr

(0.7533889716758513, 4.457925401443882e-13)

In [11]:
#Step 3 questions

In [12]:
result_lsa_score, result_lsa_by_section = lsa_model.evaluate_word_analogies(path_analogy)

In [13]:
result_lsa_score*100

7.480314960629922

In [14]:
with open(path+"word_test_small_set.txt", "w") as f:
    for d in result_lsa_by_section:
        if d['section'] == "Total accuracy":
            continue
        f.write(": "+d['section']+"\n")
        for t in d['correct']:
            f.write(" ".join(t)+"\n")
        for t in d['incorrect']:
            f.write(" ".join(t)+"\n")

In [15]:
result_google_score, result_google_by_section = google_model.evaluate_word_analogies(path+"word_test_small_set.txt")

In [16]:
result_google_score*100

70.07874015748031

In [17]:
semantic_lsa_correct = 0
syntactic_lsa_correct = 0

semantic_lsa_total = 0
syntactic_lsa_total = 0


for d in result_lsa_by_section:
    if d['section'] == "Total accuracy":
            continue
    if d['section'].startswith('gram'):
        syntactic_lsa_correct += len(d['correct'])
        syntactic_lsa_total += len(d['correct']) + len(d['incorrect'])
    else:
        semantic_lsa_correct += len(d['correct'])
        semantic_lsa_total += len(d['correct']) + len(d['incorrect'])
        
semantic_lsa_score = semantic_lsa_correct/semantic_lsa_total
syntactic_lsa_score = syntactic_lsa_correct/syntactic_lsa_total

In [18]:
semantic_lsa_score*100,syntactic_lsa_score*100

(20.0, 6.442444339175745)

In [19]:
#Sanity Check
(semantic_lsa_total*semantic_lsa_score + syntactic_lsa_total*syntactic_lsa_score)/(semantic_lsa_total+syntactic_lsa_total)*100

7.480314960629922

In [20]:
semantic_google_correct = 0
syntactic_google_correct = 0

semantic_google_total = 0
syntactic_google_total = 0


for d in result_google_by_section:
    if d['section'] == "Total accuracy":
            continue
    if d['section'].startswith('gram'):
        syntactic_google_correct += len(d['correct'])
        syntactic_google_total += len(d['correct']) + len(d['incorrect'])
    else:
        semantic_google_correct += len(d['correct'])
        semantic_google_total += len(d['correct']) + len(d['incorrect'])
        
semantic_google_score = semantic_google_correct/semantic_google_total
syntactic_google_score = syntactic_google_correct/syntactic_google_total

In [21]:
semantic_google_score*100,syntactic_google_score*100

(88.0, 68.59308384651823)

In [22]:
#Sanity Check
(semantic_google_total*semantic_google_score + syntactic_google_total*syntactic_google_score)/(semantic_google_total+syntactic_google_total)*100

70.07874015748031